<a href="https://colab.research.google.com/github/kmt19/siim_otani/blob/master/drive2gcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google import colab
colab.drive.mount('/content/gdrive')
!ls 'gdrive/My Drive'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
'Colab Notebooks'    dicom-images-train   手法のアイデア.gdoc
 dicom-images-test   train.csv


In [0]:
!pip install pydicom
!git clone https://github.com/kmt19/siim_otani/
!ls

     |████████████████████████████████| 7.0MB 2.8MB/s 
Cloning into 'siim_otani'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 44 (delta 19), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (44/44), done.
gdrive	sample_data  siim_otani


In [0]:
import os
import pydicom
from glob import glob
from tqdm import tqdm
import pandas as pd
from siim_otani.mask_functions import mask2rle,rle2mask
from siim_otani.pipline2 import get_dataloaders
DRIVE_DIR = 'gdrive/My Drive/'
folders = os.listdir(DRIVE_DIR)
print(folders)


['手法のアイデア.gdoc', 'dicom-images-test', 'dicom-images-train', 'Colab Notebooks', 'train.csv']


In [0]:
train_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-train')
test_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-test')
print(os.listdir(train_dicom_dir)[0])
a_dir = os.path.join(train_dicom_dir, os.listdir(train_dicom_dir)[0])
print(os.listdir(a_dir))
b_dir = os.path.join(a_dir, os.listdir(a_dir)[0])
print(os.listdir(b_dir))
"""
ここから各Directory内のlistを読み込み、
BASE_DIR = 'train_dicom_dir/%s/%s/' % (file_list, file_list2)を
for文でもらってきて最終的にpathを保存したdataframeを作成する。
"""
for filename in os.listdir(train_dicom_dir):
  BASE_DIR = 'train_dicom_dir/%s/%s/' % (filename, filename)


1.2.276.0.7230010.3.1.2.8323329.13318.1517875244.629616
['1.2.276.0.7230010.3.1.3.8323329.13318.1517875244.629615']
['1.2.276.0.7230010.3.1.4.8323329.13318.1517875244.629617.dcm']


In [0]:
import os
import pandas as pd
from tqdm import tqdm
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets.folder import pil_loader
import pydicom
from PIL import Image
from __future__ import print_function
from collections import defaultdict, deque
import datetime
import pickle
import time
import torch.distributed as dist
import errno
import collections
import numpy as np
import torch.utils.data
from torchvision import transforms
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


In [0]:
train_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-train')
test_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-test')
data_files={}
phase = ["train","test"]
data_files["train"] = pd.DataFrame(columns=['Path', 'ID'])
data_files["test"] = pd.DataFrame(columns=['Path'])
SEGMENTATION = DRIVE_DIR + 'train.csv'
anns = pd.read_csv(SEGMENTATION)
train_list=os.listdir(train_dicom_dir)

for i, filename in tqdm(enumerate(train_list)):
  train_glob = f'{train_dicom_dir}/{filename}/*/*.dcm'
  data_files["train"].loc[i] = [glob(train_glob)[0], anns.loc[i][1]]
  if i==10:
    break

test_list=os.listdir(test_dicom_dir)  
for i, filename in tqdm(enumerate(test_list)):
  test_glob = f'{test_dicom_dir}/{filename}/*/*.dcm'
  data_files["test"].loc[i] = [glob(test_glob)[0]]
  if i==10:
    break
traindataloader = get_dataloaders(data_files)


0it [00:00, ?it/s]
0it [00:00, ?it/s]


"\nfor folder in folders:\n  new_folder = path + '/' + folder\n  new_folder = new_folder + '/' + os.listdir(new_folder)[0]\n  new_file = new_folder + '/' + os.listdir(new_folder)[0]\n  ds = pydicom.dcmread(new_file)\n  break\n  "

In [0]:
def reduce_dict(input_dict, average=True):
    """
    Args:
        input_dict (dict): all the values will be reduced
        average (bool): whether to do average or sum
    Reduce the values in the dictionary from all processes so that all processes
    have the averaged results. Returns a dict with the same fields as
    input_dict, after reduction.
    """
    world_size = get_world_size()
    if world_size < 2:
        return input_dict
    with torch.no_grad():
        names = []
        values = []
        # sort the keys so that they are consistent across processes
        for k in sorted(input_dict.keys()):
            names.append(k)
            values.append(input_dict[k])
        values = torch.stack(values, dim=0)
        dist.all_reduce(values)
        if average:
            values /= world_size
        reduced_dict = {k: v for k, v in zip(names, values)}
    return reduced_dict
  
  def warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor):

    def f(x):
        if x >= warmup_iters:
            return 1
        alpha = float(x) / warmup_iters
        return warmup_factor * (1 - alpha) + alpha

    return torch.optim.lr_scheduler.LambdaLR(optimizer, f)

def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

g
g
g
g
g
g
g
g
g



KeyboardInterrupt



In [0]:
# create mask rcnn model
num_classes = 2
device = torch.device('cuda:0')

model_ft = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
in_features = model_ft.roi_heads.box_predictor.cls_score.in_features
model_ft.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
in_features_mask = model_ft.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
model_ft.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
model_ft.to(device)

for param in model_ft.parameters():
    param.requires_grad = True
params = [p for p in model_ft.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=5,gamma=0.1)

num_epochs = 6
for epoch in range(num_epochs):
    train_one_epoch(model_ft, optimizer, traindataloader["train"], device, epoch, print_freq=100)
    lr_scheduler.step()